In [131]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
import sys

In [133]:
sys.path.append("../")
sys.path.append("../../")

In [134]:
from CCG import utils
from rahul_code.CCG_new import constants
import rahul_code.CCG_new.utils as r_utils 
from nltk.ccg import chart, lexicon
import re

In [177]:
mini_sent = """
The phrase \"'s grandmother\" occurs between SUBJ and OBJ
"""

mini_sent_2 = """
The phrase 'angry' is in the sentence
"""

longer_sent = """
The phrase \"'s grandmother\" occurs between SUBJ and OBJ and there are no more than four words between SUBJ and OBJ .
"""

In [178]:
sent = mini_sent_2

In [179]:
pre_process = utils.pre_process_sent(sent)

In [180]:
pre_process

['The', 'phrase', "'angry'", '$In', 'the', 'sentence']

In [181]:
sent_tokenized = utils.print_tokenized(pre_process)

In [182]:
sent_tokenized

[['$The', '$Word', "'angry'", '$In', '$The', '$Sentence']]

In [155]:
raw_lexicon = constants.RAW_GRAMMAR
raw_lexicon = r_utils.add_rules_to_grammar(['"angry"'], raw_lexicon)

In [156]:
raw_lexicon

':- S,NP,N,PP\n        VP :: S\\NP\n        Det :: NP/N\n        Adj :: N/N\n        arg => NP {None}\n        #$True => (S\\VP)/PP {None}\n        #$False => (S\\VP)/PP {None}\n        $And => var\\.,var/.,var {\\x y.\'@And\'(x,y)}\n        $Or => var\\.,var/.,var {\\x y.\'@Or\'(x,y)}\n        $Not => (S\\NP)\\(S\\NP) {None}\n        $Not => (S\\NP)/(S\\NP) {None}\n        $All => NP/N {None}\n        $All => NP {None}\n        $All => NP/NP {None}\n        $Any => NP/N {None}\n        $None => N {None}\n        #$Is => (S\\NP)/NP {\\y x.\'@Is\'(x,y)}\n        #$Is => (S\\NP)/(S\\NP) {\\y x.\'@Is\'(x,y)}\n        $Is => (S\\NP)/PP {\\y x.\'@Is\'(x,y)}   # word \'a\' occurs between <S> and <O>\n        $Is => (S\\NP)\\PP {\\y x.\'@Is\'(x,y)}  # between <S> and <O> occurs word \'a\'\n        $Is => (S\\PP)\\NP {\\x y.\'@Is\'(x,y)}  # between <S> and <O> word \'a\' occurs\n        $Exists => S\\NP/PP {\\y x.\'@Is\'(x,y)}\n        #$Exists => S\\NP {None}\n        $Int => Adj {None} #Ther

In [157]:
lex = lexicon.fromstring(raw_lexicon, True)
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)

In [193]:
sent_tokenized = [['$The', '$Word', '"angry"', '$In', '$The', '$Sentence']]

In [194]:
logic_forms = []

In [195]:
for i, e in enumerate(sent_tokenized):
    parses = list(parser.parse(e))
    logic_forms += parses

In [196]:
len(logic_forms)

28

In [197]:
raw_sems = list(set([str(tree.label()[0].semantics()) for tree in logic_forms]))

In [198]:
for sem in raw_sems:
    print(sem)
    print("\n")

'@In0'('Sentence','@Word'("angry"))




In [79]:
sem_split = re.split(',|(\()', raw_sems[0])

In [80]:
sem_split = [token for token in sem_split if token]

In [81]:
new_sem = []
for i,w in enumerate(sem_split):
    if w=='(':
        new_sem.insert(-1,'(')
    else:
        new_sem.append(sem_split[i])

In [82]:
# for duo in zip(sem_split, new_sem):
#     print(duo)

In [83]:
print(new_sem)

['(', "'@In0'", "'Sentence'", '(', "'@Word'", '"angry"))']


In [57]:
# s = new_sem[6]
# print(s)
# sem_replace = s.replace('SPACE', ' ')
for com in constant.comma_index:
    sem_replace = sem_replace.replace('COMMA' + str(constant.comma_index[com]), com)
if sem_replace.startswith("'"):
    if sem_replace.endswith(")"):
        posi = len(sem_replace)-1
        while sem_replace[posi]==")":
            posi-=1
        assert sem_replace[posi]=="\'"
    else:
        posi = len(sem_replace)-1
        assert sem_replace[posi] == "\'"
    print(sem_replace)
    sem_replace = sem_replace[0]+sem_replace[1:posi].replace('\'','\\\'')+sem_replace[posi:]

NameError: name 's' is not defined

In [84]:
new_sem

['(', "'@In0'", "'Sentence'", '(', "'@Word'", '"angry"))']

In [85]:
sem_ = ''
for i,s in enumerate(new_sem):
    sem_replace = s.replace('SPACE', ' ')
#     for com in constants.comma_index:
#         sem_replace = sem_replace.replace('COMMA' + str(constants.comma_index[com]), com)
    if sem_replace.startswith("\""):
        if sem_replace.endswith(")"):
            posi = len(sem_replace)-1
            while sem_replace[posi]==")":
                posi-=1
            assert sem_replace[posi]=="\""
        else:
            posi = len(sem_replace)-1
            assert sem_replace[posi] == "\""

        sem_replace = sem_replace[0]+sem_replace[1:posi].replace('\"','\\\"')+sem_replace[posi:]
    if new_sem[i-1]=='(':
        sem_=sem_+sem_replace
    else:
        sem_ = sem_+','+sem_replace

In [86]:
sem_

',(\'@In0\',\'Sentence\',(\'@Word\',"angry"))'

In [87]:
temp = (".root", eval(sem_[1:]))

In [88]:
temp

('.root', ('@In0', 'Sentence', ('@Word', 'angry')))

In [224]:
hehe = { temp : 1}

In [95]:
('.root', eval("('@In0','Sentence',('@Word',\"angry\"))"))

('.root', ('@In0', 'Sentence', ('@Word', 'angry')))

In [108]:
parsed_sems = [utils.parse_sem(str(tree.label()[0].semantics())) for tree in logic_forms]

In [109]:
parsed_sems[0]

('.root',
 ('@Is', ('@Word', "'s grandmother"), ('@between', ('@And', 'ArgY', 'ArgX'))))

In [65]:
parsed_sems[1]

('.root',
 ('@Is', ('@Word', "'s grandmother"), ('@between', ('@And', 'ArgY', 'ArgX'))))

In [66]:
type(parsed_sems[1]), len(parsed_sems[1]), len(parsed_sems[1][0]), len(parsed_sems[1][1])

(tuple, 2, 5, 3)

In [67]:
parsed_sems[1][0], type(parsed_sems[1][1])

('.root', tuple)

In [68]:
parsed_sems[1]

('.root',
 ('@Is', ('@Word', "'s grandmother"), ('@between', ('@And', 'ArgY', 'ArgX'))))

In [69]:
parsed_sems[1][1]

('@Is', ('@Word', "'s grandmother"), ('@between', ('@And', 'ArgY', 'ArgX')))

In [70]:
parsed_sems[1][1][1]

('@Word', "'s grandmother")

In [71]:
parsed_sems[1][1][2]

('@between', ('@And', 'ArgY', 'ArgX'))

In [28]:
labeling_functions = [utils.recurse(parsed_sem) for parsed_sem in parsed_sems]

In [238]:
labeling_functions[0]()

AttributeError: 'str' object has no attribute 'get_mid'

In [229]:
parsed_sems[0]

('.root',
 ('@Is', ('@Word', "'s grandmother"), ('@between', ('@And', 'ArgY', 'ArgX'))))

In [230]:
def collect_features(semantics, features):
    if isinstance(semantics, tuple):

        for child in semantics[1:]:
            if isinstance(child, tuple) and child[0] != semantics[0]:
                features.append((semantics[0],child[0]))
            collect_features(child, features)

In [231]:
opsfeature = {'@And@Is': 0, '@Is@Word': 1, '@Is@Direct': 2, '@Direct@Left0': 3, '.root@And': 4, '@Is@between': 5, '@Direct@Right0': 6, '@Is@And': 7, '@And@Word': 8, '@between@And': 9, '@Word@And': 10, '@Left0@Word': 11, '@Is@Left0': 12, '@Right0@Word': 13, '@between@Word': 14, '.root@Is': 15, '@Is@Right0': 16, '@Right0@And': 17, '@Left0@And': 18, '@Is@LessThan': 19, '@LessThan@between': 20, '@LessThan@Num': 21, '@And@Right0': 22, '@And@between': 23, '.root@between': 24, '@between@Is': 25, '.root@Left': 26, '@Left@Word': 27, '@And@Left0': 28}


In [232]:
import numpy as np

In [233]:
feature_vector = np.zeros([len(opsfeature),1],np.float32)

In [235]:
op_feature = []

In [236]:
collect_features(parsed_sems[0], op_feature)

In [237]:
op_feature

[('.root', '@Is'), ('@Is', '@Word'), ('@Is', '@between'), ('@between', '@And')]